In [1]:
import pandas as pd 
import numpy as np
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_

                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')
    
    return df[features]

In [2]:
df_train = pd.read_csv('train.csv')
df_ = generate_features(df_train)

In [3]:
X = df_.values
Y = df_train['target'].values

X = X[np.isfinite(Y)]
Y = Y[np.isfinite(Y)]

index = np.arange(len(X))

In [5]:
import catboost as cbt 

model = cbt.CatBoostRegressor(
    iterations=500,                 # Number of boosting iterations
    learning_rate=0.1,              # Learning rate
    depth=6,                        # Depth of the trees
    loss_function='RMSE',           # Loss function to optimize
    eval_metric='MAE',             # Metric for evaluation
    random_seed=42) 

model.fit(X, Y, verbose=10)

0:	learn: 6.3915959	total: 926ms	remaining: 7m 41s
10:	learn: 6.3222996	total: 7.91s	remaining: 5m 51s
20:	learn: 6.3065113	total: 13.4s	remaining: 5m 5s
30:	learn: 6.3003832	total: 18.9s	remaining: 4m 46s
40:	learn: 6.2969929	total: 24.4s	remaining: 4m 33s
50:	learn: 6.2927937	total: 29.7s	remaining: 4m 21s


KeyboardInterrupt: 